In [31]:
import io
import time
import requests
import datetime
import pandas as pd

from util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'
path_data = './data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)

In [32]:
# Define today, yesterday and day before yesterday dates
date_tday = datetime.date.today()
date_yday = Functions.get_previous_date(date_tday)
date_bf_yday = Functions.get_previous_date(date_yday)
date_tday, date_yday, date_bf_yday

(datetime.date(2020, 11, 7),
 datetime.date(2020, 11, 6),
 datetime.date(2020, 11, 5))

In [33]:
# Today url for regions file
url_tday = Functions.get_url_date(level, date_tday)
# Yesterday url for regions file
url_yday = Functions.get_url_date(level, date_yday)
# Day before yesterday url for regions file
url_bf_yday = Functions.get_url_date(level, date_bf_yday)

url_tday, url_yday, url_bf_yday

('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201107.csv',
 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201106.csv',
 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201105.csv')

In [34]:
wait = True
if wait:
    while True:
        response = requests.get(url_tday)
        if response.status_code == 200:
            print('Presenti dati odierni')
            date_last = date_tday
            csv = response.content
            df_raw_r_1 = pd.read_csv(io.StringIO(csv.decode('utf-8')))
            df_raw_r_0 = pd.read_csv(url_yday)
            break
        else:
            print('Status code: {}. Retrying...'.format(status_code))
        time.sleep(20)
else:
    print('Dati di ieri')
    date_last = date_yday
    df_raw_r_1 = pd.read_csv(url_yday)
    df_raw_r_0 = pd.read_csv(url_bf_yday)

Presenti dati odierni


In [35]:
# State S
df_raw_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-11-07T17:00:00,ITA,1,Piemonte,45.073274,7.680687,4122,284,4406,47083,...,3573,4437,37812,4594,40806,53089,93895,1134670,715993,NaN
1,2020-11-07T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,154,14,168,2013,...,13,117,1697,202,3584,496,4080,44845,27267,NaN
2,2020-11-07T17:00:00,ITA,3,Lombardia,45.466794,9.190347,5813,610,6423,121206,...,10263,11489,108581,18226,197252,57184,254436,3218458,1981508,NaN
3,2020-11-07T17:00:00,ITA,5,Veneto,45.434905,12.338452,1300,176,1476,46181,...,3720,3815,25682,2568,25971,49936,75907,2437080,944143,NaN
4,2020-11-07T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,286,45,331,7029,...,605,872,6422,442,12386,1838,14224,576329,240788,NaN
5,2020-11-07T17:00:00,ITA,7,Liguria,44.411493,8.932699,1303,77,1380,10728,...,416,1092,21522,1914,24746,10798,35544,481315,246833,NaN
6,2020-11-07T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,1723,188,1911,33738,...,1919,2009,28619,4781,48031,21018,69049,1715527,906435,In seguito a verifica sui dati comunicati nei ...
7,2020-11-07T17:00:00,ITA,9,Toscana,43.769231,11.255889,1368,214,1582,40001,...,2143,2787,17352,1532,48875,11592,60467,1198493,791081,NaN
8,2020-11-07T17:00:00,ITA,10,Umbria,43.106758,12.388247,345,58,403,8967,...,485,688,4731,178,4029,10250,14279,325727,187400,NaN
9,2020-11-07T17:00:00,ITA,11,Marche,43.616760,13.518875,454,67,521,9306,...,587,631,7415,1045,18037,250,18287,342550,201310,NaN


In [36]:
# State S - 1
df_raw_r_0.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-11-06T17:00:00,ITA,1,Piemonte,45.073274,7.680687,3871,268,4139,43777,...,3942,4878,36993,4549,39162,50296,89458,1117280,703606,NaN
1,2020-11-06T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,154,13,167,2001,...,16,129,1600,195,3481,482,3963,44359,27094,NaN
2,2020-11-06T17:00:00,ITA,3,Lombardia,45.466794,9.190347,5563,570,6133,111233,...,4842,9934,107463,18118,190116,52831,242947,3172359,1960088,NaN
3,2020-11-06T17:00:00,ITA,5,Veneto,45.434905,12.338452,1184,169,1353,42584,...,3218,3297,25612,2543,25565,46527,72092,2417660,937280,NaN
4,2020-11-06T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,260,44,304,6451,...,376,542,6162,435,11675,1677,13352,567637,238032,NaN
5,2020-11-06T17:00:00,ITA,7,Liguria,44.411493,8.932699,1315,78,1393,10299,...,469,1127,20881,1879,24320,10132,34452,475685,244325,NaN
6,2020-11-06T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,1673,177,1850,31880,...,1754,1953,28559,4752,46964,20077,67041,1695309,897938,In seguito a verifica sui dati comunicati nei ...
7,2020-11-06T17:00:00,ITA,9,Toscana,43.769231,11.255889,1303,209,1512,37928,...,1958,2592,16738,1502,46541,11139,57680,1180350,780994,effettuati 19323 tamponi rapidi antigenici di ...
8,2020-11-06T17:00:00,ITA,10,Umbria,43.106758,12.388247,326,53,379,8506,...,496,767,4532,174,3841,9750,13591,320893,185144,NaN
9,2020-11-06T17:00:00,ITA,11,Marche,43.616760,13.518875,444,60,504,8736,...,613,697,7379,1037,17400,256,17656,338947,199235,NaN


In [37]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-11-07T17:00:00,ITA,1,Piemonte,45.073274,7.680687,4122,284,4406,47083,...,3573,4437,37812,4594,40806,53089,93895,1134670,715993,NaN
1,2020-11-07T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,154,14,168,2013,...,13,117,1697,202,3584,496,4080,44845,27267,NaN
2,2020-11-07T17:00:00,ITA,3,Lombardia,45.466794,9.190347,5813,610,6423,121206,...,10263,11489,108581,18226,197252,57184,254436,3218458,1981508,NaN
3,2020-11-07T17:00:00,ITA,4,Trentino-Alto Adige,46.382635,11.204676,604,54,658,9440,...,619,1024,12150,801,18280,4769,23049,560589,246764,NaN
4,2020-11-07T17:00:00,ITA,5,Veneto,45.434905,12.338452,1300,176,1476,46181,...,3720,3815,25682,2568,25971,49936,75907,2437080,944143,NaN
5,2020-11-07T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,286,45,331,7029,...,605,872,6422,442,12386,1838,14224,576329,240788,NaN
6,2020-11-07T17:00:00,ITA,7,Liguria,44.411493,8.932699,1303,77,1380,10728,...,416,1092,21522,1914,24746,10798,35544,481315,246833,NaN
7,2020-11-07T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,1723,188,1911,33738,...,1919,2009,28619,4781,48031,21018,69049,1715527,906435,In seguito a verifica sui dati comunicati nei ...
8,2020-11-07T17:00:00,ITA,9,Toscana,43.769231,11.255889,1368,214,1582,40001,...,2143,2787,17352,1532,48875,11592,60467,1198493,791081,NaN
9,2020-11-07T17:00:00,ITA,10,Umbria,43.106758,12.388247,345,58,403,8967,...,485,688,4731,178,4029,10250,14279,325727,187400,NaN


In [38]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,11,2020-11-06T17:00:00,ITA,1,Piemonte,45.073274,7.680687,3871,268,4139,...,3942,4878,36993,4549,39162,50296,89458,1117280,703606,NaN
1,17,2020-11-06T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,154,13,167,...,16,129,1600,195,3481,482,3963,44359,27094,NaN
2,8,2020-11-06T17:00:00,ITA,3,Lombardia,45.466794,9.190347,5563,570,6133,...,4842,9934,107463,18118,190116,52831,242947,3172359,1960088,NaN
3,19,2020-11-06T17:00:00,ITA,4,Trentino-Alto Adige,46.382635,11.204676,582,52,634,...,-405,974,11757,789,17436,4589,22025,554506,244625,NaN
4,18,2020-11-06T17:00:00,ITA,5,Veneto,45.434905,12.338452,1184,169,1353,...,3218,3297,25612,2543,25565,46527,72092,2417660,937280,NaN
5,5,2020-11-06T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,260,44,304,...,376,542,6162,435,11675,1677,13352,567637,238032,NaN
6,7,2020-11-06T17:00:00,ITA,7,Liguria,44.411493,8.932699,1315,78,1393,...,469,1127,20881,1879,24320,10132,34452,475685,244325,NaN
7,4,2020-11-06T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,1673,177,1850,...,1754,1953,28559,4752,46964,20077,67041,1695309,897938,In seguito a verifica sui dati comunicati nei ...
8,15,2020-11-06T17:00:00,ITA,9,Toscana,43.769231,11.255889,1303,209,1512,...,1958,2592,16738,1502,46541,11139,57680,1180350,780994,effettuati 19323 tamponi rapidi antigenici di ...
9,16,2020-11-06T17:00:00,ITA,10,Umbria,43.106758,12.388247,326,53,379,...,496,767,4532,174,3841,9750,13591,320893,185144,NaN


In [39]:
# Print raw columns
# print(df_r_1.columns, df_r_0.columns)
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rf_1 = df_r_1.drop(cols_to_drop, axis=1).sort_values(by='codice_regione').reset_index(drop=True)
df_rf_0 = df_r_0.drop(cols_to_drop, axis=1).sort_values(by='codice_regione').reset_index(drop=True)
# df_rf_1.columns, df_rf_0.columns

In [40]:
df_rf_0 = df_rf_0.sort_values(by='codice_regione').reset_index(drop=True)
df_rf_0;

In [41]:
df_r = df_rf_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-11-07T17:00:00,1,Piemonte,4122,284,4406,47083,51489,3573,4437,37812,4594,40806,53089,93895,1134670,715993,4341375
1,2020-11-07T17:00:00,2,Valle d'Aosta,154,14,168,2013,2181,13,117,1697,202,3584,496,4080,44845,27267,125501
2,2020-11-07T17:00:00,3,Lombardia,5813,610,6423,121206,127629,10263,11489,108581,18226,197252,57184,254436,3218458,1981508,10103969
3,2020-11-07T17:00:00,4,Trentino-Alto Adige,604,54,658,9440,10098,619,1024,12150,801,18280,4769,23049,560589,246764,1074819
4,2020-11-07T17:00:00,5,Veneto,1300,176,1476,46181,47657,3720,3815,25682,2568,25971,49936,75907,2437080,944143,4907704
5,2020-11-07T17:00:00,6,Friuli Venezia Giulia,286,45,331,7029,7360,605,872,6422,442,12386,1838,14224,576329,240788,1211357
6,2020-11-07T17:00:00,7,Liguria,1303,77,1380,10728,12108,416,1092,21522,1914,24746,10798,35544,481315,246833,1543127
7,2020-11-07T17:00:00,8,Emilia-Romagna,1723,188,1911,33738,35649,1919,2009,28619,4781,48031,21018,69049,1715527,906435,4467118
8,2020-11-07T17:00:00,9,Toscana,1368,214,1582,40001,41583,2143,2787,17352,1532,48875,11592,60467,1198493,791081,3722729
9,2020-11-07T17:00:00,10,Umbria,345,58,403,8967,9370,485,688,4731,178,4029,10250,14279,325727,187400,880285


In [42]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-11-07T17:00:00,1,Piemonte,4122,284,4406,47083,51489,3573,4437,37812,4594,40806,53089,93895,1134670,715993,4341375,367
1,2020-11-07T17:00:00,2,Valle d'Aosta,154,14,168,2013,2181,13,117,1697,202,3584,496,4080,44845,27267,125501,20
2,2020-11-07T17:00:00,3,Lombardia,5813,610,6423,121206,127629,10263,11489,108581,18226,197252,57184,254436,3218458,1981508,10103969,983
3,2020-11-07T17:00:00,4,Trentino-Alto Adige,604,54,658,9440,10098,619,1024,12150,801,18280,4769,23049,560589,246764,1074819,106
4,2020-11-07T17:00:00,5,Veneto,1300,176,1476,46181,47657,3720,3815,25682,2568,25971,49936,75907,2437080,944143,4907704,825
5,2020-11-07T17:00:00,6,Friuli Venezia Giulia,286,45,331,7029,7360,605,872,6422,442,12386,1838,14224,576329,240788,1211357,175
6,2020-11-07T17:00:00,7,Liguria,1303,77,1380,10728,12108,416,1092,21522,1914,24746,10798,35544,481315,246833,1543127,209
7,2020-11-07T17:00:00,8,Emilia-Romagna,1723,188,1911,33738,35649,1919,2009,28619,4781,48031,21018,69049,1715527,906435,4467118,516
8,2020-11-07T17:00:00,9,Toscana,1368,214,1582,40001,41583,2143,2787,17352,1532,48875,11592,60467,1198493,791081,3722729,415
9,2020-11-07T17:00:00,10,Umbria,345,58,403,8967,9370,485,688,4731,178,4029,10250,14279,325727,187400,880285,70


In [43]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [44]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_rf_1['ricoverati_con_sintomi'] - df_rf_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_rf_1['terapia_intensiva'] - df_rf_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_rf_1['totale_ospedalizzati'] - df_rf_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_rf_1['isolamento_domiciliare'] - df_rf_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_rf_1['nuovi_positivi'] - df_rf_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_rf_1['tamponi'] - df_rf_0['tamponi']
df_r['nuovi_casi_testati'] = df_rf_1['casi_testati'] - df_rf_0['casi_testati']
df_r['nuovi_deceduti'] = df_rf_1['deceduti'] - df_rf_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_rf_1['dimessi_guariti'] - df_rf_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-11-07T17:00:00,3,Lombardia,5813,610,6423,121206,127629,10263,11489,...,62.05,9973,1555,46099,21420,108,1118,24.92,0.11371,2.52
1,2020-11-07T17:00:00,1,Piemonte,4122,284,4406,47083,51489,3573,4437,...,77.38,3306,-441,17390,12387,45,819,25.51,0.10220,2.16
2,2020-11-07T17:00:00,15,Campania,1756,179,1935,63571,65506,3310,4309,...,41.92,3232,-199,22696,17659,15,984,18.99,0.07447,1.42
3,2020-11-07T17:00:00,5,Veneto,1300,176,1476,46181,47657,3720,3815,...,21.33,3597,518,19420,6863,25,70,19.64,0.07773,1.55
4,2020-11-07T17:00:00,9,Toscana,1368,214,1582,40001,41583,2143,2787,...,51.57,2073,195,18143,10087,30,614,15.36,0.07486,1.62
5,2020-11-07T17:00:00,12,Lazio,2619,237,2856,46180,49036,2352,2618,...,31.73,2241,-81,30644,26458,27,239,8.54,0.04463,1.08
6,2020-11-07T17:00:00,8,Emilia-Romagna,1723,188,1911,33738,35649,1919,2009,...,36.43,1858,56,20218,8497,29,60,9.94,0.04497,1.55
7,2020-11-07T17:00:00,19,Sicilia,1161,169,1330,19407,20737,1224,1363,...,31.41,1210,-60,8431,5601,35,104,16.17,0.02743,0.61
8,2020-11-07T17:00:00,7,Liguria,1303,77,1380,10728,12108,416,1092,...,36.84,429,-35,5630,2508,35,641,19.40,0.07077,2.30
9,2020-11-07T17:00:00,16,Puglia,876,124,1000,15573,16573,944,1054,...,33.88,853,108,7081,7063,14,96,14.88,0.02630,0.62


In [45]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)